# Environmental day Web Scrapping - Data Prepare

In [17]:
import time 
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

## Scrapping from UN NEWS

In [67]:
MAIN_URL = "https://news.un.org/"

In [80]:
articles_url = []

In [86]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.maximize_window()
time.sleep(1)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/hermione/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/var/folders/fj/3rtd_zxj3fd8fx2vq1knfgdc0000gn/T/ipykernel_75154/1983766380.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
for i in range(1,93):
    browser.get(f"https://news.un.org/en/news/topic/climate-change?page={i}") 
    get_page = bs(browser.page_source)
    for stories in get_page('div',{'class':'view-content'})[0]('h1',{'class':'story-title'}):
        articles_url.append(stories('a')[0]['href'])
    print(f"---Finished page{i}---")
    time.sleep(10)

---Finished page1---
---Finished page2---
---Finished page3---
---Finished page4---
---Finished page5---
---Finished page6---
---Finished page7---
---Finished page8---
---Finished page9---
---Finished page10---
---Finished page11---
---Finished page12---
---Finished page13---
---Finished page14---
---Finished page15---
---Finished page16---
---Finished page17---
---Finished page18---
---Finished page19---
---Finished page20---
---Finished page21---
---Finished page22---
---Finished page23---
---Finished page24---
---Finished page25---
---Finished page26---
---Finished page27---
---Finished page28---
---Finished page29---
---Finished page30---
---Finished page31---
---Finished page32---
---Finished page33---


In [79]:
print(articles_url)

['/en/story/2022/05/1119042', '/en/story/2022/05/1118942', '/en/story/2022/05/1118932', '/en/story/2022/05/1118902', '/en/story/2022/05/1118862', '/en/story/2022/05/1118742', '/en/story/2022/05/1118722', '/en/story/2022/05/1118602', '/en/story/2022/05/1118452', '/en/story/2022/05/1118262']
